In [ ]:
import xarray as xr
import glob
from xgcm import Grid
import matplotlib.pyplot as plt
import cmocean
import cartopy.crs as ccrs
%pylab inline
import warnings
warnings.simplefilter('ignore') # filter some warning messages

In [ ]:
in_file = 'roho160_clim_GLORYS_20170115_to_20191130.nc'
out_file = 'roho160_clim_GLORYS_20170115_to_20191130_zarr.nc'

In [ ]:
# functions 
# modified from 
# https://raphaeldussin.medium.com/modern-python-tools-for-the-roms-ocean-model-bfca8642db01

def remove_all_small_variables(ds):
    """ remove all the variables that have less than 3 dimensions
    
        Parameters:
            ds (xarray.Dataset): ROMS dataset
    """
    todrop = ['pm','pn','z_r','z_w','Tcline','theta_s','theta_b'] #,'angle'] # sol: had to select them by hand
    for v in todrop:
        ds = ds.drop_vars(v)
    return ds

def select_interior(ds):
    """
    discard "exterior" u,v,rho-points to build a symetric grid
    
        Parameters:
            ds (xarray.Dataset): ROMS dataset
    """
    ds = ds.isel(xi_rho=slice(1,-1), eta_rho=slice(1,-1))
    if 'xi_v' in ds.dims:
        ds = ds.isel(xi_v=slice(1,-1))
    if 'eta_u' in ds.dims:
        ds = ds.isel(eta_u=slice(1,-1))
    return ds

def rename_dims(ds):
    """ rename dimensions
    
        Parameters:
            ds (xarray.Dataset): ROMS dataset
    
    """
    ds = ds.rename({'xi_rho': 'xh', 'xi_v': 'xh', 'xi_u': 'xq','xi_psi': 'xq',
                    'eta_rho': 'yh', 'eta_v': 'yq', 'eta_u': 'yh', 'eta_psi': 'yq',
                    'ocean_time': 'time'
                    })
    return ds

def add_coords(ds):
    """ set coordinate variables as xarray coordinates
    
        Parameters:
            ds (xarray.Dataset): ROMS dataset
    """
    ds = ds.set_coords(['Cs_r', 'Cs_w', 'hc', 'h', 'time',
                        'lon_rho', 'lon_v', 'lon_u', 'lon_psi',
                        'lat_rho', 'lat_v', 'lat_u', 'lat_psi'])
    return ds

In [ ]:
# open file 
ds = xr.open_dataset(in_file)
ds = remove_all_small_variables(ds)
ds = select_interior(ds)
ds = rename_dims(ds)
ds = add_coords(ds)
ds = ds.chunk({'time': 1}) # zarr
# save in zarr format
ds.to_zarr(out_file, consolidated=True, mode='w')

ds